In [2]:
import dill
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

In [5]:
class TransformerFechas(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        columna_fecha = pd.to_datetime(X["pickup_datetime"])
        fecha_df = pd.DataFrame()
        fecha_df["weekday"] = columna_fecha.dt.weekday
        fecha_df["hour"] = columna_fecha.dt.hour
        return fecha_df

class TransformerDistancia(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_init = X[["pickup_latitude", "pickup_longitude"]].to_numpy()
        X_final = X[["dropoff_latitude", "dropoff_longitude"]].to_numpy()

        # Distancia de Haversine
        distancia = self.distancia_haversine(X_init=X_init, X_final=X_final)
        distancia_df = pd.DataFrame()
        distancia_df["distancia"] = distancia
        return distancia_df
    file_name = 'taxi-trip-duration.csv'
    def distancia_haversine(self, X_init, X_final):
        # Convertir de decimal a radianes
        X_init = np.radians(X_init)
        X_final = np.radians(X_final)

        # haversine formula 
        dlat = X_final[:, 0] - X_init[:, 0] 
        dlon = X_final[:, 1] - X_init[:, 1]
        a = np.sin(dlat / 2) ** 2 + np.cos(X_init[:, 0]) * np.cos(X_final[:, 0]) * np.sin(dlon / 2) ** 2
        c = 2 * np.arcsin(np.sqrt(a))
        r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
        return c * r

In [6]:
file_name = 'taxi-trip-duration.csv'
try:
    data = pd.read_csv(file_name)
    print(f'{file_name} found on disk')
except:
    url = "https://factored-workshops.s3.amazonaws.com/taxi-trip-duration.csv"
    print(f'{file_name} not found on disk, downloading from{url}')
    data = pd.read_csv(url)
    data.to_csv(file_name, index=False)

data = pd.read_csv("https://factored-workshops.s3.amazonaws.com/taxi-trip-duration.csv")
# Limitar rango de datos
tiempo_minimo = 60 # 1 minuto
tiempo_maximo = 36000 # 10 horas
data = data[
    (data["trip_duration"] > tiempo_minimo) &
    (data["trip_duration"] < tiempo_maximo)
]

y = data["trip_duration"]
selected_columns = [
    'vendor_id',
    'pickup_datetime',
    'passenger_count',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'pickup_borough',
    'dropoff_borough'
]
input_df = data[selected_columns]
train_df, val_df, y_train, y_val = train_test_split(input_df, y, random_state=0)

taxi-trip-duration.csv found on disk
